## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Country
0,0,grand gaube,-20.0064,57.6608,73.80,74,8,21.68,light rain,MU
1,1,bluff,-46.6000,168.3333,55.22,73,18,13.06,few clouds,NZ
2,2,princeton,25.5384,-80.4089,81.01,88,1,5.75,clear sky,US
3,3,trairi,-3.2778,-39.2689,77.58,78,3,9.75,clear sky,BR
4,4,butaritari,3.0707,172.7902,83.19,74,91,7.90,overcast clouds,KI


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would ilke for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))


What is the minimum temperature you would ilke for your trip?70
What is the maximum temperature you would like for your trip?90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Country
0,0,grand gaube,-20.0064,57.6608,73.80,74,8,21.68,light rain,MU
2,2,princeton,25.5384,-80.4089,81.01,88,1,5.75,clear sky,US
3,3,trairi,-3.2778,-39.2689,77.58,78,3,9.75,clear sky,BR
4,4,butaritari,3.0707,172.7902,83.19,74,91,7.90,overcast clouds,KI
5,5,atuona,-9.8000,-139.0333,79.88,79,25,18.03,light rain,PF
6,6,hithadhoo,-0.6000,73.0833,83.93,71,100,17.56,overcast clouds,MV
12,12,nagua,19.3832,-69.8474,77.45,87,100,9.13,overcast clouds,DO
13,13,olinda,-8.0089,-34.8553,74.93,78,40,6.87,scattered clouds,BR
18,18,honiara,-9.4333,159.9500,86.09,67,95,7.87,overcast clouds,SB
21,21,souillac,-20.5167,57.5167,73.38,78,14,15.68,few clouds,MU


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                282
City                   282
Lat                    282
Lng                    282
Max Temp               282
Humidity               282
Cloudiness             282
Wind Speed             282
Current Description    282
Country                282
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()
clean_df.head(5)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Country
0,0,grand gaube,-20.0064,57.6608,73.80,74,8,21.68,light rain,MU
2,2,princeton,25.5384,-80.4089,81.01,88,1,5.75,clear sky,US
3,3,trairi,-3.2778,-39.2689,77.58,78,3,9.75,clear sky,BR
4,4,butaritari,3.0707,172.7902,83.19,74,91,7.90,overcast clouds,KI
5,5,atuona,-9.8000,-139.0333,79.88,79,25,18.03,light rain,PF


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,grand gaube,MU,73.80,light rain,-20.0064,57.6608,
2,princeton,US,81.01,clear sky,25.5384,-80.4089,
3,trairi,BR,77.58,clear sky,-3.2778,-39.2689,
4,butaritari,KI,83.19,overcast clouds,3.0707,172.7902,
5,atuona,PF,79.88,light rain,-9.8000,-139.0333,
6,hithadhoo,MV,83.93,overcast clouds,-0.6000,73.0833,
12,nagua,DO,77.45,overcast clouds,19.3832,-69.8474,
13,olinda,BR,74.93,scattered clouds,-8.0089,-34.8553,
18,honiara,SB,86.09,overcast clouds,-9.4333,159.9500,
21,souillac,MU,73.38,few clouds,-20.5167,57.5167,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
    }

#6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"]=f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
   
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
        print("Hotel not found... skipping")

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [9]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,grand gaube,MU,73.80,light rain,-20.0064,57.6608,Veranda Paul et Virginie Hotel & Spa
2,princeton,US,81.01,clear sky,25.5384,-80.4089,American Best Inn
3,trairi,BR,77.58,clear sky,-3.2778,-39.2689,janelas do mar
4,butaritari,KI,83.19,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
5,atuona,PF,79.88,light rain,-9.8000,-139.0333,Villa Enata


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,grand gaube,MU,73.80,light rain,-20.0064,57.6608,Veranda Paul et Virginie Hotel & Spa
2,princeton,US,81.01,clear sky,25.5384,-80.4089,American Best Inn
3,trairi,BR,77.58,clear sky,-3.2778,-39.2689,janelas do mar
4,butaritari,KI,83.19,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
5,atuona,PF,79.88,light rain,-9.8000,-139.0333,Villa Enata
...,...,...,...,...,...,...,...
648,gainesville,US,75.54,light rain,29.6516,-82.3248,"Holiday Inn Gainesville-University Ctr, an IHG..."
651,sabzevar,IR,85.59,clear sky,36.2126,57.6819,پزشکی قانونی سبزوار
652,santa ana,US,84.79,few clouds,33.7456,-117.8678,Santa Ana Travel Inn
655,tual,ID,81.86,scattered clouds,-5.6667,132.7500,Grand Vilia Hotel


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [13]:
max_temps=hotel_df["Max Temp"]

In [16]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temps, dissipating=False,
                                 max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))